# Домашнее задание:
## Сравнение детекторов на примере детекции игроков в спорте

Необходимо обучить на целевом датасете (игроки в спорте) два типа детекторов: 1) Detectron2 (семейтсво R-CNN) и 2) Ultralytics (семейство YOLO). После обучения нужно замерить метрику mAP для разных моделей и сравнить результаты. Задание будет принятно только после выполнения всех 5 задач. Также необходимо продемонстрировать код и сводную таблицу с результатами.

### Задачи:
1) Обучить любую модель из [Detectron2](https://github.com/facebookresearch/detectron2) детектировать игроков в баскетбольном матче. Число классов: 2 (две команды). В качестве начальной модели взять любую из [предобученных моделей](https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md) (Faster R-CNN или RetinaNet). Перед обучением, необходимо привести целевой датасет к формату detectron2. Пример того, как обучать Detectron2 на своем датасете можно найти [здесь](https://colab.research.google.com/drive/1Xdm2KrBauT0lEyKFEiLie7nhzXajgEkH?usp=sharing). Посчитать метрику COCO mAP.

2) Обучить любую модель из [Ultralytics](https://github.com/ultralytics/ultralytics) (YOLOv8, YOLOv9, YOLOv10). В качестве начальной модели необходимо взять предобученную модель (например, yolov8n.pt). Необходимо также адаптировать целевой датасет к формату Ultralytics. Пример того, как обучить YOLOv8 на своем датасете можно найти [здесь](https://colab.research.google.com/drive/1fPZT1d-EBuYdDgXAmfnlKhkFilRnrjGj?usp=sharing). Посчитать метрику COCO mAP.

3) Обучить модель из Ultralytics с нуля (from scratch), аналогично п.2.

4) Протестировать модель "из коробки" из Ultralytics (т.е. без дообучения) и посчитать mAP для тестовой выборки.

5) Свести в единую таблицу результаты всех моделей (по метрике mAP), сравнить и сделать выводы (точность, скорость обучения, эффект предобучения, удобство работы с фреймворком (detectron2 и ultralitics).

### Датасет:
Скачайте датасет [otus_basketball_dataset](https://disk.yandex.ru/d/thdvaAKQS3ASXA) и распакуйте. Внутри находятся папки `images_train` (400 изображений) и `images_test` (100 изображений), а также файлы с аннотациями к ним: `anno_train.json` и `anno_test.json`. Файлы с аннотациями имеет следующую структуру:
```
image_name: {
     player_id: {
             'box': [x,y,w,h],
             'team': int
     }
     .....
}
....
```
Здесь `image_name` - имя изображения в соответствующей папке,<br>
`player_id` - id игрока в текущем изображении (в данном задании id не важен),<br>
`'box': [x,y,w,h]` - bounding box соответствующего игрока (координаты нормализованы от 0 до 1, поэтому их нужно домножить на размеры изображения, чтобы перевести их в целые координаты фрейма),<br>
`'team': int` - id команды к которой принадлежит игрок  в данном кадре (может принимать значения 0 или 1).

In [15]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone https://github.com/facebookresearch/detectron2
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Cloning into 'detectron2'...
No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8'
The system cannot find the file specified.


In [17]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:42:46_Pacific_Standard_Time_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0
torch:  2.0 ; cuda:  cu117
detectron2: 0.6


In [31]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

ModuleNotFoundError: No module named 'fvcore'